In [46]:
import networkx as nx
import pandas as pd

import altair as alt
import altair_saver
import nx2alt

# Data
Here I set up fake data for this viz, normally I'd be grabbing from an API

In [2]:
# arbitrary social graph skeleton:
G = nx.karate_club_graph()

In [52]:
# random data helpers
from string import ascii_lowercase
from random import sample, choice, randrange
import datetime
from collections import defaultdict

def random_date(start_date=datetime.date(2015, 1, 1),
                end_date=datetime.date(2021, 2, 1)):
    time_between_dates = end_date - start_date
    days_between_dates = time_between_dates.days
    random_number_of_days = randrange(days_between_dates)
    return str(start_date + datetime.timedelta(days=random_number_of_days))

In [56]:
# random paper-y data:
firstnames = ['Alice', 'Wilmer', 'Bert', 'Jim', 'Sally', 'Pepper', 'Winnie', 'Gerty', 'Stan', 'Oswald', 'Penny', 'Owen', 'Primo', 'Lucy', 'Ben', 'Shaquille', 'Frank', 'Fyodor', 'Ivan']
lastnames = ['Tomson', 'Spear', 'Quinn', 'Wallace', 'Pillmore', 'Stone', 'Black', 'Whitmore', 'Olson', 'Knowles', 'Shakur', 'Smalls', 'Nonce', 'Salisbury', 'Weir', 'Piu']
def random_name(): return f'{choice(firstnames)} {choice(lastnames)}'
def random_username(): return ''.join(choice(ascii_lowercase) for n in range(5))

# emails:
domain_map = {
    'Mr. Hi': 'hi.edu',
    'Officer': 'officer.edu'
}
def inverted_map(mapping): return {v:k for k,v in mapping.items()}
def org2dom(org): return domain_map.get(org, 'unknown')
def dom2org(dom): return inverted_map(domain_map).get(dom, 'unknown')
def eml2dom(eml): return eml.split('@')[-1]
def eml2org(eml): return dom2org(eml2dom(eml))

def alnumspace(string): return string.isalnum() or string==' '
stopwords = "i|me|my|myself|we|us|our|ours|ourselves|you|your|yours|yourself|yourselves|he|him|his|himself|she|her|hers|herself|it|its|itself|they|them|their|theirs|themselves|what|which|who|whom|whose|this|that|these|those|am|is|are|was|were|be|been|being|have|has|had|having|do|does|did|doing|will|would|should|can|could|ought|i'm|you're|he's|she's|it's|we're|they're|i've|you've|we've|they've|i'd|you'd|he'd|she'd|we'd|they'd|i'll|you'll|he'll|she'll|we'll|they'll|isn't|aren't|wasn't|weren't|hasn't|haven't|hadn't|doesn't|don't|didn't|won't|wouldn't|shan't|shouldn't|can't|cannot|couldn't|mustn't|let's|that's|who's|what's|here's|there's|when's|where's|why's|how's|a|an|the|and|but|if|or|because|as|until|while|of|at|by|for|with|about|against|between|into|through|during|before|after|above|below|to|from|up|upon|down|in|out|on|off|over|under|again|further|then|once|here|there|when|where|why|how|all|any|both|each|few|more|most|other|some|such|no|nor|not|only|own|same|so|than|too|very|say|says|said|shall"
stopwords = set(stopwords.upper().split('|'))
wordlist = "Declarative visualization grammars can accelerate development, facilitate retargeting across platforms, and allow language-level optimizations. However, existing declarative visualization languages are primarily concerned with visual encoding, and rely on imperative event handlers for interactive behaviors. In response, we introduce a model of declarative interaction design for data visualizations. Adopting methods from reactive programming, we model low-level events as composable data streams from which we form higher-level semantic signals. Signals feed predicates and scale inversions, which allow us to generalize interactive selections at the level of item geometry (pixels) into interactive queries over the data domain. Production rules then use these queries to manipulate the visualization’s appearance. To facilitate reuse and sharing, these constructs can be encapsulated as named interactors: standalone, purely declarative specifications of interaction techniques. We assess our model’s feasibility and expressivity by instantiating it with extensions to the Vega visualization grammar. Through a diverse range of examples, we demonstrate coverage over an established taxonomy of visualization interaction techniques.We present Reactive Vega, a system architecture that provides the first robust and comprehensive treatment of declarative visual and interaction design for data visualization. Starting from a single declarative specification, Reactive Vega constructs a dataflow graph in which input data, scene graph elements, and interaction events are all treated as first-class streaming data sources. To support expressive interactive visualizations that may involve time-varying scalar, relational, or hierarchical data, Reactive Vega’s dataflow graph can dynamically re-write itself at runtime by extending or pruning branches in a data-driven fashion. We discuss both compile- and run-time optimizations applied within Reactive Vega, and share the results of benchmark studies that indicate superior interactive performance to both D3 and the original, non-reactive Vega system.We present Vega-Lite, a high-level grammar that enables rapid specification of interactive data visualizations. Vega-Lite combines a traditional grammar of graphics, providing visual encoding rules and a composition algebra for layered and multi-view displays, with a novel grammar of interaction. Users specify interactive semantics by composing selections. In Vega-Lite, a selection is an abstraction that defines input event processing, points of interest, and a predicate function for inclusion testing. Selections parameterize visual encodings by serving as input data, defining scale extents, or by driving conditional logic. The Vega-Lite compiler automatically synthesizes requisite data flow and event handling logic, which users can override for further customization. In contrast to existing reactive specifications, Vega-Lite selections decompose an interaction design into concise, enumerable semantic units. We evaluate Vega-Lite through a range of examples, demonstrating succinct specification of both customized interaction methods and common techniques such as panning, zooming, and linked selection."
wordlist = [c for c in filter(alnumspace, wordlist)]
wordlist = ''.join(wordlist).split(' ')
wordlist = [word.upper() for word in wordlist if (word and word.upper() not in stopwords)]

topics = {word: choice(range(10)) for word in set(wordlist)}

d = defaultdict(list)
for word, topic in topics.items():
    d[topic] += [word]
topic_data = pd.DataFrame(d.items()).rename(columns={0:'topic', 1:'topic_words'})

links = ['https://www.google.com', 'https://www.nytimes.com', 'https://www.altair-viz.github.io']

In [6]:
from collections import Counter
# analytics
def guess_org(emls):
    return Counter([eml2org(eml) for eml in emls]).most_common(n=1)[0][0]

## Node Data

In [9]:
for node in G.nodes:
    n = G.nodes[node]
    # change club to org
    try: n['org'] = n.pop('club')
    except: pass  # might already be popped
    n['name'] = random_name()
    n['email'] = f'{random_username()}@{org2dom(n['org'])}'
    
    # analytic
    n['centrality'] = nx.degree_centrality(G)[node]

## Edge Data

In [10]:
for edge in G.edges:
    e = G.edges[edge]
    e['title'] = ' '.join(choice(wordlist) for n in range(3))
    e['keywords'] = [choice(wordlist).lower() for n in range(5)]
    e['emails'] = [G.nodes[author]['email'] for author in edge]
    e['link'] = choice(links)
    e['date'] = random_date()
    
    # analytic
    e['topic'] = topics[choice(e['keywords']).upper()]
    e['org'] = guess_org(e['emails'])

# Viz

In [11]:
node_data, edge_data = nx2alt.altair_network_data(G, nx.spring_layout)

In [192]:
#selection brushes
timebrush = alt.selection_interval(encodings=['x'])
node_selector = alt.selection_single(empty='none')
legend_selector = alt.selection_multi(fields=['topic'], bind='legend')

node_base, edge_base = nx2alt.base_layers(node_data, edge_data)

edges = (
    edge_base
    .mark_line(opacity=.9)
    .encode(
        color=alt.condition(timebrush, 
                            alt.Color('topic:N', legend=None),
                            alt.value('lightgray')),
        strokeWidth=alt.condition(timebrush, alt.value(3), alt.value(1))
    )
)

nodes = (
    node_base
    .mark_circle(opacity=.9)
    .encode(
        fill='org:N',
        size=alt.Size(
            'centrality:Q', 
            scale=alt.Scale(range=[200,1000]),
            legend=None),
        tooltip=['name', 'email', 'org'],
    )
    .add_selection(node_selector)  # hack to make tooltips work
)

labels = node_base.mark_text().encode(text='node_id',)

coauthor_network = (
    (edges + nodes + labels)
    .properties(
        width=1325,
        height=520
    )
    .interactive()
    .add_selection(legend_selector)
    
)

topic_timestrip = (
    topic_strip
    .encode(
        x='date:T',
        color=alt.condition(timebrush, 'topic:N', alt.value('lightgray')),
        tooltip=['title', 'keywords'],
        href='link:N',
    )
    .add_selection(timebrush)
    .properties(width=1275)
)

legend = (
    topic_strip
    .encode(
        color='topic:N',
        tooltip=['topic_words:N']
    )
    .transform_lookup(
        lookup='topic',
        from_=alt.LookupData(
            data=topic_data, 
            key='topic', 
            fields=['topic_words'])
    )
    .add_selection(legend_selector)
    .properties(
        title='topic',
        width=10
    )
)

In [193]:
chart = (
    (coauthor_network & (legend | topic_timestrip))
    .configure(title=alt.TitleConfig(fontSize=12))
    .configure_legend(orient='top-left')
    .configure_axisX(title=None)
    .configure_axisY(title=None, ticks=False)
)

altair_saver.save(chart, "viz.html", inline=True) 

Questions:
Who is involved
Where are they from
What are their selectors


Who are the most important/prolific researchers?
Who would be able to access the most others

What are they researching?
How have research topics changed over time
Who are the most current authors
Are they stovepiped?

